# Tokenizer known as akshar

This notebook demonstrates **every feature** of akshar, the linguistically-aware tokenizer for Hindi, Sanskrit, and Hinglish.

## What You'll See:
- akshar segmentation (conjunct preservation)
- Code-switch detection
- Text normalization
- Hinglish processing
- Sanskrit handling
- And much more...


## Setup

Import the tokenizer and helper functions


In [60]:
from akshar import aksharTokenizer

In [61]:
tokenizer = aksharTokenizer()


In [62]:
text = "aaj मौसम बहुत अच्छा है"
tokens = tokenizer.tokenize(text)
print(tokens)

['a', 'a', 'j', ' ', 'मौ', 'स', 'म', ' ', 'ब', 'हु', 'त', ' ', 'अ', 'च्छा', ' ', 'है']


## Word-Level Tokenization

Get word-level tokens (like IndicNLP) using akshar's preprocessing

In [63]:
# This is what we have 

from indicnlp import common
common.set_resources_path("indic_nlp_resources")

In [64]:
from indicnlp.tokenize.indic_tokenize import trivial_tokenize

sentence = "नास॑दासी॒न्नो सदा॑सीत्त॒दानीं॒ नासी॒द्रजो॒ नो वयो॑मा प॒रो यत्।।"
print(trivial_tokenize(sentence))

['नास॑दासी॒न्नो', 'सदा॑सीत्त॒दानीं॒', 'नासी॒द्रजो॒', 'नो', 'वयो॑मा', 'प॒रो', 'यत्', '।', '।']


In [65]:
# This is what we offer 

from akshar import word_tokenize_hindi,word_tokenize,word_tokenize_sanskrit

# test it
text_hi = "aaj मौसम बहुत अच्छा है"
text_sa = "नास॑दासी॒न्नो सदा॑सीत्त॒दानीं॒ नासी॒द्रजो॒ नो वयो॑मा प॒रो यत्।"
text_idk = "मौसम बहुत what सदा॑सीत्त॒दानीं॒"
text = text_hi


In [66]:
hi = word_tokenize_hindi(text_hi)
sa = word_tokenize_hindi(text_sa)
ikik = word_tokenize_hindi(text_idk)

print(f"Hindi : {hi} \n Sanskrit : {sa} \n mix : {ikik} \n")

Hindi : ['aaj', 'मौसम', 'बहुत', 'अच्छा', 'है'] 
 Sanskrit : ['नास॑दासी॒न्नो', 'सदा॑सीत्त॒दानीं॒', 'नासी॒द्रजो॒', 'नो', 'वयो॑मा', 'प॒रो', 'यत्', '।'] 
 mix : ['मौसम', 'बहुत', 'what', 'सदा॑सीत्त॒दानीं॒'] 



In [67]:
analysis = tokenizer.explain(text)
print(analysis)

{'original': 'aaj मौसम बहुत अच्छा है', 'normalized': 'aaj मौसम बहुत अच्छा है', 'akshars': ['a', 'a', 'j', ' ', 'मौ', 'स', 'म', ' ', 'ब', 'हु', 'त', ' ', 'अ', 'च्छा', ' ', 'है'], 'code_switches': [('aaj ', 'roman'), ('मौसम बहुत अच्छा है', 'devanagari')], 'tokens': ['a', 'a', 'j', ' ', 'मौ', 'स', 'म', ' ', 'ब', 'हु', 'त', ' ', 'अ', 'च्छा', ' ', 'है'], 'stats': {'akshar_count': 16, 'script_switches': 1, 'devanagari_ratio': 0.8181818181818182, 'roman_ratio': 0.18181818181818182}}


# akshar Features Demo

This notebook demonstrates all the intelligent features of akshar tokenizer.


## 1. Basic Tokenization

Tokenize text without any trained model (falls back to akshar-level)


In [68]:
examples = [
    "आज मौसम बहुत अच्छा है",
    "नमस्ते दुनिया",
    "क्षेत्रे धर्मक्षेत्रे",
]

for text in examples:
    tokens = tokenizer.tokenize(text)
    print(f"Text: {text}")
    print(f"Tokens ({len(tokens)}): {' | '.join(tokens)}")
    print()


Text: आज मौसम बहुत अच्छा है
Tokens (15): आ | ज |   | मौ | स | म |   | ब | हु | त |   | अ | च्छा |   | है

Text: नमस्ते दुनिया
Tokens (7): न | म | स्ते |   | दु | नि | या

Text: क्षेत्रे धर्मक्षेत्रे
Tokens (7): क्षे | त्रे |   | ध | र्म | क्षे | त्रे



## 2. akshar Segmentation

Shows how Devanagari conjuncts stay together as single units


In [69]:
from akshar import segment_akshars

sanskrit_examples = [
    "क्षेत्र",      # kShetra - notice क्ष stays together
    "ज्ञान",        # gyaan - ज्ञ stays together
    "त्रिशूल",      # trishul - त्र stays together
    "धर्मक्षेत्रे",  # dharmakshetre - multiple conjuncts
]

for text in sanskrit_examples:
    akshars = segment_akshars(text)
    print(f"Text: {text}")
    print(f"akshars: [ {' ] [ '.join(akshars)} ]")
    print(f"Count: {len(akshars)}")
    print()


Text: क्षेत्र
akshars: [ क्षे ] [ त्र ]
Count: 2

Text: ज्ञान
akshars: [ ज्ञा ] [ न ]
Count: 2

Text: त्रिशूल
akshars: [ त्रि ] [ शू ] [ ल ]
Count: 3

Text: धर्मक्षेत्रे
akshars: [ ध ] [ र्म ] [ क्षे ] [ त्रे ]
Count: 4



## 3. Code-Switch Detection

Detects boundaries where script changes (Hinglish!)


In [70]:
from akshar import detect_code_switches

hinglish_examples = [
    "aaj मौसम बहुत अच्छा है",
    "मैं California में रहता हूं",
    "hello यार kya हाल है",
    "आज का day बहुत nice था",
]

for text in hinglish_examples:
    switches = detect_code_switches(text)
    print(f"Text: {text}")
    print("Script Boundaries:")
    for segment, script in switches:
        print(f"  [{script:12}] '{segment}'")
    print()


Text: aaj मौसम बहुत अच्छा है
Script Boundaries:
  [roman       ] 'aaj '
  [devanagari  ] 'मौसम बहुत अच्छा है'

Text: मैं California में रहता हूं
Script Boundaries:
  [devanagari  ] 'मैं '
  [roman       ] 'California '
  [devanagari  ] 'में रहता हूं'

Text: hello यार kya हाल है
Script Boundaries:
  [roman       ] 'hello '
  [devanagari  ] 'यार '
  [roman       ] 'kya '
  [devanagari  ] 'हाल है'

Text: आज का day बहुत nice था
Script Boundaries:
  [devanagari  ] 'आज का '
  [roman       ] 'day '
  [devanagari  ] 'बहुत '
  [roman       ] 'nice '
  [devanagari  ] 'था'



## 4. Text Normalization

Handles Unicode, semantic normalization, and Hinglish cleaning


In [71]:
from akshar import normalize_text

normalization_examples = [
    "Hello नमस्ते WORLD",           # mixed case Roman
    "Heyyy यार kya HAAL hai",       # elongations + case
    "मैं California में रहता हूं",  # mixed scripts
]

print("Text Normalization (lowercase Roman, preserve Devanagari):\n")
for text in normalization_examples:
    normalized = normalize_text(text)
    print(f"Original:   {text}")
    print(f"Normalized: {normalized}")
    print()


Text Normalization (lowercase Roman, preserve Devanagari):

Original:   Hello नमस्ते WORLD
Normalized: hello नमस्ते world

Original:   Heyyy यार kya HAAL hai
Normalized: hey यार kya haal hai

Original:   मैं California में रहता हूं
Normalized: मैं california में रहता हूं



## 5. Hinglish Normalization

Handles elongations and variations common in social media


In [72]:
from akshar.normalize import remove_elongations, roman_phonetic_signature

hinglish_variations = [
    "heyyy",
    "yaaaaar", 
    "niceeee",
    "bohoooot",
]

print("Elongation Removal:\n")
for word in hinglish_variations:
    cleaned = remove_elongations(word)
    print(f"{word:15} → {cleaned}")

print("\n" + "="*50)
print("\nPhonetic Signatures (for alignment):\n")

variations = [
    ["nahi", "nahii", "nahee"],
    ["yaar", "yar"],
    ["achha", "acha", "accha"],
]

for group in variations:
    signatures = [roman_phonetic_signature(w) for w in group]
    print(f"Variations: {', '.join(group)}")
    print(f"Signatures: {', '.join(signatures)}")
    print()


Elongation Removal:

heyyy           → hey
yaaaaar         → yar
niceeee         → nice
bohoooot        → bohot


Phonetic Signatures (for alignment):

Variations: nahi, nahii, nahee
Signatures: nahi, nahii, nahi

Variations: yaar, yar
Signatures: yar, yar

Variations: achha, acha, accha
Signatures: acha, aca, acca



## 6. Detailed Analysis

The `explain()` method shows complete breakdown


In [73]:
text = "yaar aaj ka मौसम बहुत अच्छा hai"
analysis = tokenizer.explain(text)

print(f"Original Text: {analysis['original']}")
print(f"Normalized:    {analysis['normalized']}")
print(f"\nakshars ({len(analysis['akshars'])}): {' | '.join(analysis['akshars'][:20])}...")
print(f"\nTokens ({len(analysis['tokens'])}): {' | '.join(analysis['tokens'][:20])}...")

print(f"\nCode Switches:")
for segment, script in analysis['code_switches']:
    print(f"  [{script:12}] '{segment}'")

print(f"\nStatistics:")
stats = analysis['stats']
print(f"  akshar count:    {stats['akshar_count']}")
print(f"  Script switches:  {stats['script_switches']}")
print(f"  Devanagari ratio: {stats['devanagari_ratio']:.1%}")
print(f"  Roman ratio:      {stats['roman_ratio']:.1%}")


Original Text: yaar aaj ka मौसम बहुत अच्छा hai
Normalized:    yaar aaj ka मौसम बहुत अच्छा hai

akshars (26): y | a | a | r |   | a | a | j |   | k | a |   | मौ | स | म |   | ब | हु | त |  ...

Tokens (26): y | a | a | r |   | a | a | j |   | k | a |   | मौ | स | म |   | ब | हु | त |  ...

Code Switches:
  [roman       ] 'yaar aaj ka '
  [devanagari  ] 'मौसम बहुत अच्छा '
  [roman       ] 'hai'

Statistics:
  akshar count:    26
  Script switches:  2
  Devanagari ratio: 51.6%
  Roman ratio:      48.4%


### Feature 18: Morphological Segmentation

Break words into morphemes (requires: pip install morfessor)


In [74]:
from akshar import get_hindi_segmenter, segment_hindi

segmenter = get_hindi_segmenter()

if segmenter.is_model_loaded():
    print("✓ Hindi Morfessor model loaded!\n")
    print("Morphological Segmentation:\n")
    
    # hindi_words = ["नमस्ते", "अच्छा", "समझना", "लड़की", "करना"]
    hindi_words = [
        "खाना", "पीना", "देखना", "चलना", "सोना",
        "सुनना", "खेलना", "लिखना", "पढ़ना", "सीखना",
        "गाना", "हँसना", "रोना", "सोचना", "बोलना",

        "पढ़ाई", "लड़ाई", "मिट्टी", "जल्दी", "भूख",
        "भय", "काबू", "कहानी", "खुशी", "उदासी",
        "मजदूर", "महिला", "आदमी", "कबूतर", "तितली",

        "चुनौती", "उपयोग", "अनुभव", "चिकित्सा", "नागरिक",
        "मूल्यांकन", "आवश्यकता", "अज्ञानता", "व्यवस्था", "परिभाषा",
        "संस्कृति", "विकास", "समर्पण", "विश्वास", "समस्या",

        "उत्तरदायित्व", "स्वतंत्रता", "नागरिकता", "प्रबंधक", "अधिकारिता",
        "दायित्वहीनता", "अनुशासनहीनता", "व्यवस्थापन", "निर्वाचन", "संगठन",
        "कर्मयोगी", "संवैधानिक", "अधिग्रहण", "अभियंत्रण", "अनुभूतिकरण"
    ]

    
    for word in hindi_words:
        morphemes = segmenter.segment_word(word)
        print(f"  {word:15} → {' + '.join(morphemes)}")
else:
    print("⚠ Morfessor not installed")
    print("Install with: pip install morfessor")
    print("\nNote: Morphological segmentation will fall back to basic mode")
    # 75% accuracy i think for startig its good i wil keep training better mdl


✓ Hindi Morfessor model loaded!

Morphological Segmentation:

  खाना            → खाना
  पीना            → पी + ना
  देखना           → देख + ना
  चलना            → चल + ना
  सोना            → सोना
  सुनना           → सुन + ना
  खेलना           → खेल + ना
  लिखना           → लिख + ना
  पढ़ना           → पढ़ + ना
  सीखना           → सीख + ना
  गाना            → गा + ना
  हँसना           → हँस + ना
  रोना            → रो + ना
  सोचना           → सोच + ना
  बोलना           → बोल + ना
  पढ़ाई           → पढ़ाई
  लड़ाई           → लड़ाई
  मिट्टी          → मिट्टी
  जल्दी           → जल्द + ी
  भूख             → भूख
  भय              → भय
  काबू            → काबू
  कहानी           → कहानी
  खुशी            → खुशी
  उदासी           → उदास + ी
  मजदूर           → मजदूर
  महिला           → महिला
  आदमी            → आदमी
  कबूतर           → कबूतर
  तितली           → तितली
  चुनौती          → चुनौती
  उपयोग           → उपयोग
  अनुभव           → अनुभव
  चिकित्सा        → चिकित्सा
  नागरिक          

### Feature 20: Complete Feature Comparison

akshar vs Basic Tokenization


## 7. Composition Analysis

Analyze text composition across different types


In [75]:
from akshar.segment import analyze_text_composition

test_texts = {
    "Pure Hindi": "आज मौसम बहुत अच्छा है",
    "Pure English": "today weather is very nice",
    "Hinglish": "aaj ka मौसम बहुत nice hai",
    "Sanskrit": "क्षेत्रे धर्मक्षेत्रे समवेता युयुत्सवः",
}

print("Text Composition Analysis:\n")
print(f"{'Type':<15} {'akshars':<10} {'Switches':<10} {'Devanagari':<12} {'Roman':<10}")
print("="*65)

for name, text in test_texts.items():
    comp = analyze_text_composition(text)
    print(f"{name:<15} {comp['akshar_count']:<10} {comp['script_switches']:<10} "
          f"{comp['devanagari_ratio']:<12.1%} {comp['roman_ratio']:<10.1%}")


Text Composition Analysis:

Type            akshars    Switches   Devanagari   Roman     
Pure Hindi      15         0          100.0%       0.0%      
Pure English    26         0          0.0%         100.0%    
Hinglish        23         2          40.0%        60.0%     
Sanskrit        17         0          100.0%       0.0%      


## 8. Tokenization with Metadata

Get tokens plus all analysis in one call


In [76]:
text = "hello नमस्ते दोस्तों"
result = tokenizer.tokenize(text, return_metadata=True)

print("Tokenization with Metadata:\n")
print(f"Original:       {result['original_text']}")
print(f"Normalized:     {result['normalized_text']}")
print(f"Token count:    {result['token_count']}")
print(f"akshar count:  {result['akshar_count']}")
print(f"Script switches: {result['script_switches']}")
print(f"Tokens:         {' | '.join(result['tokens'])}")


Tokenization with Metadata:

Original:       hello नमस्ते दोस्तों
Normalized:     hello नमस्ते दोस्तों
Token count:    12
akshar count:  12
Script switches: 1
Tokens:         h | e | l | l | o |   | न | म | स्ते |   | दो | स्तों


## 9. Script Detection

Identify which script each character belongs to


In [77]:
from akshar.segment import identify_script

test_chars = ['न', 'a', 'क', 'h', '5', '.', 'म', 'Z']

print("Character Script Detection:\n")
for char in test_chars:
    script = identify_script(char)
    print(f"'{char}' → {script}")


Character Script Detection:

'न' → devanagari
'a' → roman
'क' → devanagari
'h' → roman
'5' → digit
'.' → punct
'म' → devanagari
'Z' → roman


## 10. Visual Comparison

Compare how different texts tokenize


In [78]:
import pandas as pd

comparison_texts = [
    "नमस्ते",
    "namaste", 
    "hello नमस्ते",
    "क्षेत्र",
    "yaar kya haal hai",
    "आज का day",
]

data = []
for text in comparison_texts:
    analysis = tokenizer.explain(text)
    data.append({
        'Text': text,
        'akshars': analysis['stats']['akshar_count'],
        'Tokens': len(analysis['tokens']),
        'Switches': analysis['stats']['script_switches'],
        'Dev%': f"{analysis['stats']['devanagari_ratio']:.0%}",
        'Rom%': f"{analysis['stats']['roman_ratio']:.0%}",
    })

df = pd.DataFrame(data)
print("\nTokenization Comparison:\n")
print(df.to_string(index=False))



Tokenization Comparison:

             Text  akshars  Tokens  Switches Dev% Rom%
           नमस्ते        3       3         0 100%   0%
          namaste        7       7         0   0% 100%
     hello नमस्ते        9       9         1  50%  50%
          क्षेत्र        2       2         0 100%   0%
yaar kya haal hai       17      17         0   0% 100%
        आज का day        8       8         1  67%  33%


## 11. Real-World Examples

Social media style Hinglish text


## 21. 🆕 Phonetic Analysis

Analyze phonetic properties: vowels, consonants, aspiration, nasalization


In [79]:
from akshar import get_phonetic_analyzer

analyzer = get_phonetic_analyzer()

print("Phonetic Properties of Characters:\n")

# test various characters
chars = ['क', 'ख', 'न', 'म', 'अ', 'आ']

for char in chars:
    props = analyzer.get_properties(char)
    if props:
        print(f"{char}:")
        print(f"  Type: {'Vowel' if props['is_vowel'] else 'Consonant'}")
        if props['is_consonant']:
            print(f"  Aspirated: {props['aspirated']}")
            print(f"  Voiced: {props['voiced']}")
            print(f"  Nasal: {props['nasal']}")
            place = analyzer.get_place_of_articulation(char)
            if place:
                print(f"  Place: {place}")
        print()

print("="*50)
print("\nWord Analysis:\n")

words = ["नमस्ते", "भारत", "ख़ुशी"]

for word in words:
    analysis = analyzer.analyze_word(word)
    print(f"{word:10} → Vowels: {analysis['vowels']}, "
          f"Consonants: {analysis['consonants']}, "
          f"Nasals: {analysis['nasals']}, "
          f"Aspirated: {analysis['aspirated']}")


Phonetic Properties of Characters:

क:
  Type: Consonant
  Aspirated: False
  Voiced: False
  Nasal: False
  Place: velar

ख:
  Type: Consonant
  Aspirated: True
  Voiced: False
  Nasal: False
  Place: velar

न:
  Type: Consonant
  Aspirated: False
  Voiced: True
  Nasal: True
  Place: dental

म:
  Type: Consonant
  Aspirated: False
  Voiced: True
  Nasal: True
  Place: labial

अ:
  Type: Vowel

आ:
  Type: Vowel


Word Analysis:

नमस्ते     → Vowels: 1, Consonants: 4, Nasals: 2, Aspirated: 1
भारत       → Vowels: 1, Consonants: 3, Nasals: 0, Aspirated: 1
ख़ुशी      → Vowels: 2, Consonants: 2, Nasals: 0, Aspirated: 2


## 22. 🆕 Multi-Script Detection

Detect multiple Indic scripts (not just Devanagari)


In [80]:
from akshar import identify_scripts, analyze_script

# pure devanagari
text1 = "नमस्ते भारत"
scripts1 = identify_scripts(text1)
print(f"Text: {text1}")
print(f"Scripts: {scripts1}\n")

# mixed content
text2 = "Hello नमस्ते World"
analysis2 = analyze_script(text2)
print(f"Text: {text2}")
print(f"Total chars: {analysis2['total_chars']}")
print(f"Indic chars: {analysis2['indic_chars']}")
print(f"Scripts: {analysis2['scripts']}")
print(f"Multilingual: {analysis2['is_multilingual']}\n")

print("="*50)
print("\nScript Identification Examples:\n")

test_texts = [
    "संस्कृतम्",           # Sanskrit/Devanagari
    "हिन्दी",              # Hindi
    "aaj ka din",         # Roman
    "मैं California जाता हूं",  # Mixed
]

for text in test_texts:
    scripts = identify_scripts(text)
    analysis = analyze_script(text)
    print(f"{text:30} → Scripts: {list(scripts.keys()) if scripts else ['roman']}")


Text: नमस्ते भारत
Scripts: {'devanagari': 10}

Text: Hello नमस्ते World
Total chars: 18
Indic chars: 6
Scripts: {'devanagari': 6}
Multilingual: False


Script Identification Examples:

संस्कृतम्                      → Scripts: ['devanagari']
हिन्दी                         → Scripts: ['devanagari']
aaj ka din                     → Scripts: ['roman']
मैं California जाता हूं        → Scripts: ['devanagari']


## 23. 🆕 Combined Intelligence Pipeline

Use all features together for complete analysis


## 24. 🆕 Character Deep Dive

Examine individual character properties in detail


In [81]:
from akshar.script_utils import ScriptAnalyzer

script_analyzer = ScriptAnalyzer()
phonetic = get_phonetic_analyzer()

# interesting characters
chars_to_examine = ['क', 'ख', 'न', 'म', 'अ', 'आ']

print("Deep Character Analysis:\n")
print("="*70)

for char in chars_to_examine:
    print(f"\nCharacter: {char}")
    print(f"Unicode: U+{ord(char):04X}")
    
    # unicode name
    name = script_analyzer.get_character_name(char)
    print(f"Name: {name}")
    
    # phonetic props
    props = phonetic.get_properties(char)
    if props:
        print(f"ITRANS: {props['itrans']}")
        if props['is_vowel']:
            print(f"Type: Vowel")
        elif props['is_consonant']:
            print(f"Type: Consonant")
            place = phonetic.get_place_of_articulation(char)
            if place:
                print(f"Articulation: {place}")
            print(f"Aspirated: {props['aspirated']}")
            print(f"Voiced: {props['voiced']}")
            print(f"Nasal: {props['nasal']}")
        
        if props['halanta']:
            print("⚠ Halanta (virama)")
        if props['anusvara']:
            print("⚠ Anusvara")

print("\n" + "="*70)


print("\nNote: Conjuncts like च् and ज्ञ are multiple unicode codepoints,")
print("      so they need special handling for character-level analysis.")

Deep Character Analysis:


Character: क
Unicode: U+0915
Name: DEVANAGARI LETTER KA
ITRANS: ka
Type: Consonant
Articulation: velar
Aspirated: False
Voiced: False
Nasal: False

Character: ख
Unicode: U+0916
Name: DEVANAGARI LETTER KHA
ITRANS: kha
Type: Consonant
Articulation: velar
Aspirated: True
Voiced: False
Nasal: False

Character: न
Unicode: U+0928
Name: DEVANAGARI LETTER NA
ITRANS: na
Type: Consonant
Articulation: dental
Aspirated: False
Voiced: True
Nasal: True

Character: म
Unicode: U+092E
Name: DEVANAGARI LETTER MA
ITRANS: ma
Type: Consonant
Articulation: labial
Aspirated: False
Voiced: True
Nasal: True

Character: अ
Unicode: U+0905
Name: DEVANAGARI LETTER A
ITRANS: a
Type: Vowel

Character: आ
Unicode: U+0906
Name: DEVANAGARI LETTER AA
ITRANS: A
Type: Vowel


Note: Conjuncts like च् and ज्ञ are multiple unicode codepoints,
      so they need special handling for character-level analysis.


## 25. 🆕 Linguistic Feature Extraction

Extract all linguistic features for ML/NLP tasks


In [82]:
from akshar import (
    aksharTokenizer,
    detect_code_switches,
    normalize_text,
    analyze_phonetics,
    analyze_script,
    segment_akshars
)

def extract_all_features(text):
    """
    Extract complete linguistic feature vector.
    Useful for ML models, text classification, etc.
    """
    tokenizer = aksharTokenizer()
    
    # get all analyses
    token_analysis = tokenizer.explain(text)
    script_analysis = analyze_script(text)
    phonetic_analysis = analyze_phonetics(text)
    akshars = segment_akshars(text)
    switches = detect_code_switches(text)
    
    features = {
        # basic
        'length': len(text),
        'tokens': len(token_analysis['tokens']),
        'akshars': len(akshars),
        
        # script composition
        'devanagari_ratio': token_analysis['stats']['devanagari_ratio'],
        'roman_ratio': token_analysis['stats']['roman_ratio'],
        'script_switches': len(switches),
        'is_multilingual': script_analysis['is_multilingual'],
        
        # phonetic
        'vowel_count': phonetic_analysis['vowels'],
        'consonant_count': phonetic_analysis['consonants'],
        'nasal_count': phonetic_analysis['nasals'],
        'aspirated_count': phonetic_analysis['aspirated'],
        
        # ratios
        'vowel_consonant_ratio': (
            phonetic_analysis['vowels'] / max(phonetic_analysis['consonants'], 1)
        ),
        'akshar_per_token': len(akshars) / max(len(token_analysis['tokens']), 1),
    }
    
    return features


# test it
test_sentences = [
    "नमस्ते भारत",
    "hello world",
    "aaj मौसम अच्छा है",
    "यार kya बात है"
]

print("Complete Feature Extraction:\n")
print("="*70)

import pandas as pd

features_list = []
for sent in test_sentences:
    features = extract_all_features(sent)
    features['text'] = sent
    features_list.append(features)

df = pd.DataFrame(features_list)

# reorder columns
cols = ['text', 'length', 'tokens', 'akshars', 'devanagari_ratio', 
        'roman_ratio', 'script_switches', 'vowel_count', 'consonant_count']
print(df[cols].to_string(index=False))

print("\n" + "="*70)
print("\n💡 These features can be used for:")
print("   • Text classification")
print("   • Language identification")
print("   • Code-mixing detection")
print("   • Sentiment analysis")
print("   • ML model training")


Complete Feature Extraction:

             text  length  tokens  akshars  devanagari_ratio  roman_ratio  script_switches  vowel_count  consonant_count
      नमस्ते भारत      11       7        7          1.000000     0.000000                1            2                7
      hello world      11      11       11          0.000000     1.000000                1            0                0
aaj मौसम अच्छा है      17      12       12          0.764706     0.235294                2            4                6
   यार kya बात है      14      11       11          0.714286     0.285714                3            3                5


💡 These features can be used for:
   • Text classification
   • Language identification
   • Code-mixing detection
   • Sentiment analysis
   • ML model training


In [83]:
social_media_texts = [
    "yaaaar aaj ka mausam bohot achaaaa hai",
    "मुझे traveling करना पसंद है",
    "आज का day बहुत productive था bro",
    "चलो coffee पीते हैं",
]

print("Social Media Hinglish Examples:\n")
for text in social_media_texts:
    analysis = tokenizer.explain(text)
    print(f"Original:   {text}")
    print(f"Normalized: {analysis['normalized']}")
    print(f"Tokens:     {' | '.join(analysis['tokens'][:15])}...")
    print()


Social Media Hinglish Examples:

Original:   yaaaar aaj ka mausam bohot achaaaa hai
Normalized: yar aaj ka mausam bohot acha hai
Tokens:     y | a | r |   | a | a | j |   | k | a |   | m | a | u | s...

Original:   मुझे traveling करना पसंद है
Normalized: मुझे traveling करना पसंद है
Tokens:     मु | झे |   | t | r | a | v | e | l | i | n | g |   | क | र...

Original:   आज का day बहुत productive था bro
Normalized: आज का day बहुत productive था bro
Tokens:     आ | ज |   | का |   | d | a | y |   | ब | हु | त |   | p | r...

Original:   चलो coffee पीते हैं
Normalized: चलो coffee पीते हैं
Tokens:     च | लो |   | c | o | f | f | e | e |   | पी | ते |   | हैं...



## 12. Sanskrit Text Processing

Complex conjuncts and classical texts


In [84]:
sanskrit_texts = [
    "क्षेत्रे धर्मक्षेत्रे समवेता युयुत्सवः",
    "धर्म की जय हो",
    "ज्ञान से ही मोक्ष मिलता है",
    "वेदाः ज्ञानस्य स्रोतः",
]

print("Sanskrit Text Processing:\n")
for text in sanskrit_texts:
    akshars = segment_akshars(text)
    tokens = tokenizer.tokenize(text)
    
    print(f"Text: {text}")
    print(f"akshars: {len(akshars)}, Tokens: {len(tokens)}")
    
    conjuncts = [a for a in akshars if len(a) > 1 and a.strip()]
    if conjuncts:
        print(f"Conjuncts preserved: {', '.join(conjuncts[:5])}")
    print()


Sanskrit Text Processing:

Text: क्षेत्रे धर्मक्षेत्रे समवेता युयुत्सवः
akshars: 17, Tokens: 17
Conjuncts preserved: क्षे, त्रे, र्म, क्षे, त्रे

Text: धर्म की जय हो
akshars: 9, Tokens: 9
Conjuncts preserved: र्म, की, हो

Text: ज्ञान से ही मोक्ष मिलता है
akshars: 15, Tokens: 15
Conjuncts preserved: ज्ञा, से, ही, मो, क्ष

Text: वेदाः ज्ञानस्य स्रोतः
akshars: 9, Tokens: 9
Conjuncts preserved: वे, दाः, ज्ञा, स्य, स्रो



## 13. Visualization Helpers

Color-coded output for terminal display


In [85]:
from akshar.viz import format_token_boundaries, format_akshar_boundaries

text = "aaj मौसम अच्छा है"
tokens = tokenizer.tokenize(text)
akshars = segment_akshars(text)

print("Visualization Formats:\n")
print("Token Boundaries:")
print(format_token_boundaries(text, tokens))
print("\nakshar Boundaries:")
print(format_akshar_boundaries(akshars))


Visualization Formats:

Token Boundaries:
a | a | j |   | मौ | स | म |   | अ | च्छा |   | है

akshar Boundaries:
[a] [a] [j] [ ] [मौ] [स] [म] [ ] [अ] [च्छा] [ ] [है]


## 14. Batch Processing

Process multiple texts efficiently


In [86]:
batch_texts = [
    "भारत एक महान देश है",
    "India is a great country",
    "मैं India से हूं",
    "यह बहुत interesting है",
    "sanskritam देवभाषा अस्ति",
]

print("Batch Processing Results:\n")
results = []
for text in batch_texts:
    tokens = tokenizer.tokenize(text)
    analysis = tokenizer.explain(text)
    results.append({
        'text': text,
        'tokens': len(tokens),
        'dev_ratio': analysis['stats']['devanagari_ratio']
    })

for r in results:
    print(f"{r['text']:<40} | Tokens: {r['tokens']:<3} | Dev: {r['dev_ratio']:.0%}")


Batch Processing Results:

भारत एक महान देश है                      | Tokens: 15  | Dev: 100%
India is a great country                 | Tokens: 24  | Dev: 0%
मैं India से हूं                         | Tokens: 11  | Dev: 62%
यह बहुत interesting है                   | Tokens: 20  | Dev: 45%
sanskritam देवभाषा अस्ति                 | Tokens: 18  | Dev: 54%


## 15. Summary Statistics

Get overall statistics from the tokenizer


In [87]:
print("Tokenizer Configuration:\n")
print(f"Model loaded: {tokenizer.model is not None}")
print(f"Model type: {tokenizer.model_type}")
print(f"Normalize Roman: {tokenizer.normalize_roman}")
print(f"Clean Hinglish: {tokenizer.clean_hinglish}")
print(f"Vocab size: {tokenizer.vocab_size()}")

print("\n" + "="*60)
print("\nakshar Features Demonstrated:")
print("  ✓ Basic tokenization")
print("  ✓ akshar segmentation (conjunct preservation)")
print("  ✓ Code-switch detection")
print("  ✓ Text normalization")
print("  ✓ Hinglish handling")
print("  ✓ Phonetic signatures")
print("  ✓ Detailed analysis")
print("  ✓ Composition analysis")
print("  ✓ Script detection")
print("  ✓ Metadata extraction")
print("  ✓ Sanskrit processing")
print("  ✓ Batch processing")
print("  ✓ Visualization helpers")


Tokenizer Configuration:

Model loaded: False
Model type: akshar
Normalize Roman: True
Clean Hinglish: True
Vocab size: 0


akshar Features Demonstrated:
  ✓ Basic tokenization
  ✓ akshar segmentation (conjunct preservation)
  ✓ Code-switch detection
  ✓ Text normalization
  ✓ Hinglish handling
  ✓ Phonetic signatures
  ✓ Detailed analysis
  ✓ Composition analysis
  ✓ Script detection
  ✓ Metadata extraction
  ✓ Sanskrit processing
  ✓ Batch processing
  ✓ Visualization helpers


In [88]:
from akshar import akshara_level_tokenization
akshara_level_tokenization("प्रकाश")  # ['प्र', 'का', 'श'] - halant-aware

['प्र', 'का', 'श']

In [89]:
from akshar import sandhi_aware_tokenization
sandhi_aware_tokenization("देवोऽपि")  # marks boundaries, preserves original

{'original': 'देवोऽपि', 'boundaries': [], 'marked': 'देवोऽपि'}

In [90]:
from akshar import schwa_deletion_modeling
schwa_deletion_modeling("कमल")  # annotates deletion positions

{'text': 'कमल', 'deletion_annotations': [('कमल', [])]}

In [91]:
from akshar import preserve_conjuncts
preserve_conjuncts("शक्ति")  # ['श', 'क्ति'] - preserves conjuncts

['श', 'क्ति']

In [92]:
from akshar import intelligent_anusvara_resolution
intelligent_anusvara_resolution("संकर")  # {'original': '...', 'resolved': 'सङ्कर'}

{'original': 'संकर', 'resolved': 'सङकर'}

In [93]:
from akshar import handle_chandrabindu
handle_chandrabindu("हँस")  # ['हँस'] - chandrabindu stays with vowel

['हँस']

In [94]:
from akshar import preserve_nukta
preserve_nukta("क़दम")  # preserves nukta characters

['क़द', 'म']

In [95]:
from akshar import recognize_virama_function
recognize_virama_function("क्ष")  # analyzes halant context

{'text': 'क्ष', 'halant_analysis': [(1, 'conjunct_formation', 'क्ष')]}

In [96]:
from akshar import mora_aware_segmentation
mora_aware_segmentation("गं ग")  # counts light/heavy syllables

{'syllables': ['गं', 'ग'],
 'mora_counts': [2, 1],
 'total_mora': 3,
 'light_syllables': 1,
 'heavy_syllables': 1}

In [97]:
from akshar import sanskrit_punctuation_tokenization
sanskrit_punctuation_tokenization("रामः। कृष्णः॥")  # ['रामः', '।', 'कृष्णः', '॥']

['रामः', '।', 'कृष्णः', '॥']

In [98]:
from akshar import preserve_svara_marks_feature
preserve_svara_marks_feature("अ॒ग्निः")  # preserves vedic accent marks

['अ॒ग्निः']

In [99]:
from akshar import devanagari_digit_tokenization
devanagari_digit_tokenization("१२३")  # ['१', '२', '३']

['१', '२', '३']

In [100]:
from akshar import preserve_zwj
preserve_zwj("क्‍या")  # preserves ZWJ/ZWNJ

'क्\u200dया'

In [101]:
from akshar import preserve_proper_names
preserve_proper_names("श्रीमंगलम्", names=['श्री'])  # preserves proper names

['श्री', 'मं', 'ग', 'ल', 'म्']

In [102]:
from akshar import provide_lemma_hints
provide_lemma_hints("गच्छामि", lemma_dict={'गच्छामि': 'गम्'})  # provides lemmas

{'surface_forms': ['गच्छामि'], 'lemma_hints': [('गच्छामि', 'गम्')]}

In [103]:
from akshar import preserve_orthographic_variants
preserve_orthographic_variants("ऋषि")  # preserves orthographic history

'ऋषि'

In [104]:
from akshar import transliteration_tokenization
transliteration_tokenization("क्ष", scheme='iast')  # IAST transliteration

{'original_tokens': ['क्ष'], 'transliterated': ['ka्ṣa'], 'scheme': 'iast'}

In [105]:
from akshar import visarga_condition_tokenization
visarga_condition_tokenization("रामः सः")  # analyzes visarga behavior

{'original': 'रामः सः', 'visarga_annotations': [], 'has_visarga': True}

In [106]:
from akshar import preserve_independent_vowels
preserve_independent_vowels("ओम्")  # ['ओम्'] - preserves independent vowels

['ओ', 'म्']

In [107]:
from akshar import emoji_tokenization
emoji_tokenization("शुभ रात्रि 🌙")  # ['शु', 'भ', ' ', 'रा', 'त्रि', ' ', '🌙']

['शु', 'भ', ' ', 'रा', 'त्रि', ' ', '🌙']